In [ ]:
from pathlib import Path
import requests
import zipfile
import numpy as np
import pandas as pd
import csv

data_path = Path.cwd().parent / "data"
fips_csv_path = data_path / "fips_states_2010.csv"
csv_path = data_path / "dataset" / "ejscreen_2020"

In [ ]:
download = requests.get(
    "https://gaftp.epa.gov/EJSCREEN/2020/EJSCREEN_2020_StatePctile.csv.zip",
    verify=False,
)
file_contents = download.content
zip_file_path = data_path / "tmp"
zip_file = open(zip_file_path / "downloaded.zip", "wb")
zip_file.write(file_contents)
zip_file.close()

In [ ]:
with zipfile.ZipFile(zip_file_path / "downloaded.zip", "r") as zip_ref:
    zip_ref.extractall(zip_file_path)
ejscreen_csv = data_path / "tmp" / "EJSCREEN_2020_StatePctile.csv"

In [ ]:
df = pd.read_csv(ejscreen_csv, dtype={"ID": "string"}, low_memory=False)

In [ ]:
# write nationwide csv
df.to_csv(csv_path / f"usa.csv", index=False)

In [ ]:
# write per state csvs
with open(fips_csv_path) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=",")
    line_count = 0

    for row in csv_reader:
        if line_count == 0:
            line_count += 1
        else:
            fips = row[0].strip()
            print(f"Generating data{fips} csv")
            df1 = df[df.ID.str[:2] == fips]
            # we need to name the file data01.csv for ogr2ogr csv merge to work
            df1.to_csv(csv_path / f"data{fips}.csv", index=False)